In [ ]:
# Import Dependencies and setup
import os
import pandas as pd
import time
import requests
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup
# import psycopg2   # All SQL Loads should be done in a new .ipynb file

##### Extraction

In [ ]:
# Extract CSV into pandas df
df_ca = pd.read_csv('../resources/california-history.csv')
df_ca.head()
df_ca =len(df_ca)
print(df_ca)
# df_ca.dtypes  -- used to check data type. Uncomment to check

##### Generate List of Data Fields

In [ ]:
# Create json url query and variable for data request
base_url = 'https://api.covidtracking.com/v1/states/ny/daily.json'
data_dict = requests.get(base_url).json()

print(json.dumps(data_dict, indent=4, sort_keys=True))
len(data_dict)

In [ ]:
# Set up dictionary of variables where data from api calls will be stored
ny_data = {'date':[], 'state':[], 'deaths':[], 
           'Icu hospitalized':[], 'positive cases viral':[], 'positive increase':[], 
           'test results total':[], 'test increase':[]
          }

print('Retrieving NY Covid data')
print('-' * 30)

# forLoop to collect data and append to data_dict
for data in data_dict:
#     try:  -- not working; keep to use in future
        ny_data['date'].append(data['date'])
        ny_data['state'].append(data['state'])
        ny_data['deaths'].append(data['deathIncrease'])
#         ny_data['daily_hospitalization'].append(data['hospitalizedIncrease'])
        ny_data['Icu hospitalized'].append(data['inIcuCurrently'])
        ny_data['positive cases viral'].append(data['positiveCasesViral'])
        ny_data['positive increase'].append(data['positiveIncrease'])
        ny_data['test results total'].append(data['totalTestResults'])
        ny_data['test increase'].append(data['totalTestResultsIncrease'])

        # Use timer to delay request to not exceed query limits.
        time.sleep(.075)
#     except:   -- not working; keep to use in future
#         print(f'Incomplete record for {data}. Skipping {data}.')

#         print(ny_data)  -- used to validate append to ny_data dictionary; uncomment to test

In [ ]:
# Convert panda dictionary to dataframe and export to csv
ny_df = pd.DataFrame.from_dict(ny_data)

ny_df['date'] = pd.to_datetime(ny_df['date'], format='%Y%m%d')
ny_df.to_csv('../resources/ny_data.csv')
ny_df.head()
# ny_df.dtypes  -- used to check data type. Uncomment to check

In [ ]:
# Read in Cleaned CA Data

clean_ca_df = pd.read_csv('../resources/clean_ca_data.csv', index_col=0)

clean_ca_df.head()
# clean_ca_df.dtypes

In [ ]:
# Merge CA and NY Data on Date
clean_ca_df['date'] = pd.to_datetime(clean_ca_df['date'], errors="coerce")
# CA_NY_Data = pd.merge(ny_df, clean_ca_df, how="inner", on="date")
CA_NY_Data = pd.concat([ny_df, clean_ca_df]).sort_index(kind="merge")
CA_NY_Data


In [ ]:
CA_NY_Data = CA_NY_Data.rename(columns= {"date":"Date",
                                        "state":"State",
                                        "deaths":"Deaths",
                                        "Icu hospitalized":"ICU Hospitalizations",
                                        "positive cases viral": "Positive Viral Cases",
                                        "positive increase": "Positive Increase",
                                        "test results total": "Total Test Results",
                                        "test increase": "Test Increase",
                                        })
# CA_NY_Data = CA_NY_Data.dropna()
# CA_NY_Data = CA_NY_Data.reset_index(drop = True)
CA_NY_Data.fillna(0, inplace=True)

CA_NY_Data

##### Scrape iframe

In [ ]:
# Retrieve COVID graphs from Syracuse website iframes
# Set url query path
path='https://www.syracuse.com/coronavirus-ny/'
f = urlopen(path)
html = str(f.read())
soup = BeautifulSoup(html, 'html.parser')
txt = soup.find_all('iframe')

for element in txt:
    print(element.attrs["src"])

##### Retrieve data from interactive chart

In [ ]:
# Copied-pasted https://datawrapper.dwcdn.net/ijEiy/2/, a dynamic webpage 
# into browser and inspected page. Examined Network, then XHR to get the link for CSV data.
url = 'https://static.dwcdn.net/data/ijEiy.csv?v=1623222240000'
proxies = {}
response = requests.get(url=url, proxies=proxies)
with open("../resources/covid_chart_data.csv", "wb") as f:
    f.write(response.content)